In [141]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import numpy.random
import math
import sys

In [142]:
def make_classification(r0=1,r1=3,k=1000):
    """
    Creaci ́on de los datos
    """
    X1 = [np.array([r0*np.cos(t),r0*np.sin(t)]) for t in range(0,k)]
    X2 = [np.array([r1*np.cos(t),r1*np.sin(t)]) for t in range(0,k)]
    X = np.concatenate((X1,X2))
    n,d = X.shape
    Y = np.zeros((2*k,2))
    Y[0:k] += [1,0]
    Y[k:] += [0,1]
    noise = np.array([np.random.normal(0,1,2) for i in range(n)])
    X += 0.5*noise
    return X,Y

In [143]:
X, Y = make_classification(k=10)
x_train, x_eval, y_train, y_eval = train_test_split(X, Y, test_size=0.3)
x_train,y_train ;

In [144]:
class Node():
    """Nodo super clase con funciones generales"""
    def __init__(self, values):
    # Agrega los par ́ametros necesarios
        self.values = values
        self.grads = None
        return
        
    def __call__(self, *kwargs):
        return self.forward(*kwargs)

    def __str__(self):
        return str(self.values) #Valor n ́um del nodo
        
    def backward(self, consumer_grad=1):
        self.grads = consumer_grad

In [145]:
class Preactivation(Node):
    def __init__(self, parent, input_size, output_size):
        self.w = np.random.uniform(0,1,(input_size, output_size)).T
        self.b = np.random.uniform(0,1, output_size)

        # Guardamos estos parámetros
        self.input_size = input_size
        self.output_size = output_size
        self.parent = parent
        self.learning_rate = 0.1

        return None
    
    def forward(self):

        values = []
        for x in self.parent.values:
            values.append(np.dot(self.w, x) + self.b)

        self.values = np.array(values)
        return self


    def backward(self, consumer_grad):
        print(consumer_grad)
        print(self.values.T)
        # self.grad_w = np.dot(consumer_grad, self.values.T)
        self.grad_w = np.outer(consumer_grad, self.values.T).T
        print(self.grad_w)
        self.grad_b = np.dot(consumer_grad, self.values.T)

        self.update()

        self.parent.backward(consumer_grad)
        return self

    def update(self):
        self.w -= self.learning_rate * self.grad_w
        self.b -= self.learning_rate * self.grad_b
        return 

In [146]:
class Activation(Node):
    def __init__(self, preactivation_node):
        self.parent = preactivation_node
        return

    def function(self, x):
        return
    
    def derivative(self, x):
        return

    def forward(self):
        values = []

        for x in self.parent.values:
            values.append(self.function(x))

        self.values = np.array(values)
        return self

    def backward(self, consumer_grad):
        self.grad = np.multiply(consumer_grad, self.derivative(self.values[0]))
        print(self.grad)
        self.parent.backward(self.grad)
        return self

In [147]:
class Tanh(Activation):
    def function(self, x):
        return np.tanh(x)

    def derivative(self, x):
        return 1 - np.tanh(x) ** 2

In [148]:
class ReLU(Activation):
    def function(self, x):
        relu = np.zeros(len(x))
        for i in range(len(x)):
            if x[i] >= 0:
                relu[i] = x[i]
        
        return relu


    def derivative(self, x):
        relu_derivative = np.zeros(len(x))

        for i in range(len(x)):
            if x[i] >= 0:
                relu_derivative[i] = x[i]
                
        return relu_derivative

In [149]:
def kronecker_delta(x,y):
    if x == y:
        return 1
    else:
        return 0
        
class Softmax(Activation):
    def function(self, X):
        add = 0
        for x in X:
            add += np.exp(x)

        softmax_x = []
        for x in X:
            softmax_x.append( np.exp(x)/add )
        
        softmax_x = np.array(softmax_x)
        return softmax_x

    def derivative(self, x):
        n = len(x)

        deriv = np.zeros((n,n))

        for i in range(n):
            for j in range(n):
                S = self.function(x)
                deriv[i,j] = S[i] * (kronecker_delta(i,j) - S[j])

        return deriv

In [150]:
class Cross_Entropy(Node):
    # Error de clasificación binario
    def __init__(self, output_node, classes = [0,1]):
        self.parent = output_node
        self.classes = classes
        return None

    def forward(self, Y_real):
        # Definido por casos para evitar infinitos innecesarios
        epsilon = sys.float_info.epsilon

        self.real_outputs = Y_real
        add = 0
        
        # E = Σ_c y_c log(y_pred_c), donde c son las clases

        # Sumamos sobre todos los datos
        for y_pred, y_real in zip(self.parent.values, Y_real):
            # Sumamos sobre todas las clases
            for c in self.classes:
                add -= y_real[c]*np.log(y_pred[c] + epsilon)
        
        self.value = add
        
        return self
    
    def backward(self, consumer_grad = 1):
        # Esto hace la división -y_real/y_pred pero a cada valor,
        # Esto da una matriz de n x 2 donde n es el número de datos
        
        dL_df = - self.real_outputs / self.parent.values
        # dL_df = - self.real_outputs + self.parent.values

        self.grad = dL_df * consumer_grad
        # print(f"dL_df = {self.grad}")
        self.parent.backward(self.grad)
        return

In [151]:
class Neural_Network():
    def __init__(self, layers, error_node, y_real):
        self.layers = layers
        self.error = error_node
        self.y_real = y_real
    
    def forward(self):
        for layer in self.layers:
            layer.forward()
            print(layer)

        # self.error.forward(self.y_real)

    def backward(self):
        self.layers[-1].backward(1)
        # self.error.backward()  

In [152]:
# Arquitectura

initial_node = Node([x_train[0]])
pre_tanh = Preactivation(initial_node, 2, 3)
tanh_layer = Tanh(pre_tanh)

pre_relu = Preactivation(tanh_layer, 3, 4)
relu_layer = ReLU(pre_relu)

pre_soft = Preactivation(relu_layer, 4, 2)
softmax_layer = Softmax(pre_soft)

error_node = Cross_Entropy(softmax_layer)


In [153]:
archi = [pre_tanh, tanh_layer, pre_relu, relu_layer, pre_soft, softmax_layer]
nn = Neural_Network(archi, error_node, y_train)

In [154]:
nn.forward()

ValueError: setting an array element with a sequence.

In [140]:
nn.backward()

[[ 0.24851213 -0.24851213]
 [-0.24851213  0.24851213]]
[[ 0.24851213 -0.24851213]
 [-0.24851213  0.24851213]]
[[3.77494537]
 [3.46324829]]
[[ 0.93811971 -0.93811971 -0.93811971  0.93811971]
 [ 0.86065921 -0.86065921 -0.86065921  0.86065921]]


ValueError: shapes (2,2) and (1,2) not aligned: 2 (dim 1) != 1 (dim 0)